# Tests

## Tracking hooks

#### Make sure that outputs are inputs to the next layer

In [1]:
!pip install torchvision

In [2]:
import torch
from torchvision.models import resnet50
from src.monitoring import activate_model_tracking, clear_hooks

In [3]:
model = resnet50()

In [4]:
storage = {}

def add_to_storage(name, output, input):
    storage[name + '_in'] = input
    storage[name + '_out'] = output

In [5]:
hooks = activate_model_tracking(model, ['layer1', 'layer2', 'layer3', 'layer4'], add_to_storage, track_inputs=True)

In [6]:
output = model.forward(torch.randn(1,3, 250, 250))

In [7]:
assert (storage['layer1_out'] == storage['layer2_in']).all()
assert (storage['layer2_out'] == storage['layer3_in']).all()
assert (storage['layer3_out'] == storage['layer4_in']).all()

In [8]:
clear_hooks(hooks)

#### make sure that batches are written properly to storage

In [9]:
# reset the storage
storage = {}

def add_to_storage(name, output, input):
    storage[name + '_in'] = input
    storage[name + '_out'] = output

In [10]:
hooks = activate_model_tracking(model, ['layer1', 'layer2', 'layer3', 'layer4'], add_to_storage, track_inputs=True)

In [11]:
d = 10
output = model.forward(torch.randn(d, 3, 250, 250))

In [12]:
# Check batching
assert storage['layer2_out'].shape[0] == d

### make sure that the hooks are properly removed

In [13]:
model.layer1

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, ke

In [14]:
assert len(model.layer1._forward_hooks) == 1

In [15]:
clear_hooks(hooks)

In [16]:
assert len(model.layer1._forward_hooks) == 0

## Recording activity into a dataset

In [17]:
import torch
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.datasets import CIFAR10

from activity_tracking import *

In [18]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [19]:
from activity_tracking import ActivityRecorder

In [20]:
tracker = ActivityRecorder(model, ['layer1', 'layer2', 'layer3', 'layer4'], 'activitydata/')

In [21]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
cifar_train = CIFAR10('/Users/luke/Datasets/', download=True, transform=transform)

#### test data read and write 

In [22]:
!mkdir activitydata

In [23]:
dl = DataLoader(cifar_train, batch_size=10)
dl_iter = iter(dl)

i = 0
for batch in dl_iter:
    model.forward(batch[0])

    if i >=4:
        break
    i += 1

In [24]:
layer1_activity = torch.stack(tracker.buffers['layer1'])
layer2_activity = torch.stack(tracker.buffers['layer2'])
layer3_activity = torch.stack(tracker.buffers['layer3'])
layer4_activity = torch.stack(tracker.buffers['layer4'])

In [25]:
layer1_activity.shape

torch.Size([50, 256, 56, 56])

In [26]:
tracker.close()

In [27]:
activity_data = ActivityDataset('./activitydata')
dl = DataLoader(activity_data, batch_size=20)
batch = next(iter(dl))

In [28]:
# are we loading the right number of datapoints?
assert len(batch) == len(activity_data.layer_names)

In [29]:
# are we loading the correct shape of the data
assert batch[0].shape == torch.Size([20, 256, 56, 56])

In [30]:
# are we loading the data that we think we are loading 
assert (layer1_activity[0:20] == batch[0]).all()
assert (layer2_activity[0:20] == batch[1]).all()
assert (layer3_activity[0:20] == batch[2]).all()
assert (layer4_activity[0:20] == batch[3]).all()

In [31]:
!rm -r activitydata

In [32]:
# Make sure that the hooks are cleared by closing
assert not len(model.layer3._forward_hooks)

#### Input data recording

In [33]:
!mkdir activitydata

In [34]:
tracker = ActivityRecorder(model, ['layer1', 'layer2', 'layer3', 'layer4'], 'activitydata/', track_inputs=True)

In [35]:
dl = DataLoader(cifar_train, batch_size=10)
dl_iter = iter(dl)

i = 0
for batch in dl_iter:
    model.forward(batch[0])

    if i >=4:
        break
    i += 1

In [36]:
tracker.close()

In [37]:
import os
assert len(os.listdir('activitydata/')) == 9

In [38]:
!rm -r activitydata